# Sistema de Recomendação Visual por Similaridade de Imagens

Este notebook implementa um sistema de **Content-Based Image Retrieval (CBIR)**. O objetivo é recomendar produtos de um catálogo com base na sua similaridade visual com uma imagem de busca.

**Metodologia:**
1.  **Extração de Características:** Usaremos uma Rede Neural Convolucional (CNN) pré-treinada (VGG16) para 'traduzir' cada imagem em um vetor numérico (embedding) que representa sua aparência (cores, formas, texturas).
2.  **Busca por Similaridade:** Comparamos o vetor da imagem de busca com os vetores de todas as imagens do catálogo usando a métrica de Similaridade de Cosseno (*Cosine Similarity*).
3.  **Ranking:** Os produtos com os vetores mais próximos (maior similaridade) são exibidos como as melhores recomendações.

### Fase 1: Configuração do Ambiente e Montagem do Drive

Começamos instalando as bibliotecas (se necessário, embora o Colab já tenha a maioria) e conectando ao Google Drive para acessar nossos arquivos de imagem.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import os
from PIL import Image
import pickle
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

print("Bibliotecas importadas com sucesso.")

# Monta o Google Drive
drive.mount('/content/drive')

### Fase 2: Carregar o Modelo e Definir a Função de Extração

Carregamos o modelo VGG16, pré-treinado no ImageNet, e removemos sua camada final de classificação (`include_top=False`). Isso o transforma em um poderoso extrator de características visuais.

A função `extrair_features` irá encapsular todo o processo de carregar, pré-processar e extrair o vetor de uma única imagem.

In [ ]:
# Carrega o modelo VGG16 pré-treinado no ImageNet, sem a camada de classificação final
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Criamos um novo modelo que usa a base do VGG16 para extrair as características da camada 'block5_pool'
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

print("Modelo extrator de características VGG16 carregado.")

def extrair_features(image_path, model):
    """
    Carrega, pré-processa uma imagem e extrai seu vetor de características (embedding).
    """
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))
        x = np.array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x, verbose=0) # verbose=0 para não poluir a saída
        features = features.flatten()
        return features
    except Exception as e:
        print(f"Erro ao processar a imagem {image_path}: {e}")
        return None

### AÇÃO NECESSÁRIA: Prepare seu Catálogo de Imagens

Antes de rodar a próxima célula, certifique-se de que você já:
1.  Criou uma pasta no seu Google Drive, por exemplo: `/content/drive/MyDrive/catalogo_imagens/`.
2.  Fez o upload de **todas as imagens** do seu catálogo de produtos para esta pasta.
3.  Verificou se a variável `caminho_catalogo` na célula abaixo corresponde exatamente ao caminho da sua pasta.

### Fase 3: Indexação do Catálogo de Produtos

Este processo (que pode ser demorado) irá passar por cada imagem do seu catálogo, extrair seu vetor de características e salvar todos os vetores em um único arquivo. Isso só precisa ser feito uma vez (ou quando novos produtos forem adicionados).

In [ ]:
# --- CONFIGURAÇÃO DE CAMINHOS ---
# ATENÇÃO: Verifique se este caminho está correto!
caminho_base = '/content/drive/MyDrive/'
caminho_catalogo = os.path.join(caminho_base, 'catalogo_imagens/')
caminho_salvar_features = os.path.join(caminho_base, 'features_produtos.pkl')

# Lista todos os arquivos de imagem no diretório
arquivos_de_imagem = [os.path.join(caminho_catalogo, f) for f in os.listdir(caminho_catalogo) if f.endswith(('.jpg', '.png', '.jpeg'))]

catalogo_features = {}

print(f"Iniciando extração de features para {len(arquivos_de_imagem)} imagens...")

for img_path in arquivos_de_imagem:
    features = extrair_features(img_path, model)
    if features is not None:
        catalogo_features[img_path] = features
        # Opcional: imprimir um feedback a cada 10 imagens
        if len(catalogo_features) % 10 == 0:
            print(f"Processadas {len(catalogo_features)}/{len(arquivos_de_imagem)} imagens...")

# Salva o dicionário de features em um arquivo
with open(caminho_salvar_features, 'wb') as f:
    pickle.dump(catalogo_features, f)

print(f"\nExtração concluída! Features salvas em {caminho_salvar_features}")

### Fase 4: Sistema de Recomendação em Ação

Finalmente, vamos carregar nosso catálogo de features, escolher uma imagem para a busca e encontrar os produtos mais similares visualmente.

In [ ]:
# --- Carregar as features salvas ---
caminho_features = '/content/drive/MyDrive/features_produtos.pkl'
try:
    with open(caminho_features, 'rb') as f:
        catalogo_features = pickle.load(f)
except FileNotFoundError:
    print("Erro: Arquivo de features não encontrado. Certifique-se de que a Fase 3 foi executada com sucesso.")

caminhos_imagens = list(catalogo_features.keys())
vetores_features = np.array(list(catalogo_features.values()))

# --- Imagem de Busca ---
# ATENÇÃO: Coloque aqui o caminho de uma imagem que você quer usar para buscar similares.
# Pode ser uma imagem de dentro do seu catálogo ou uma nova.
caminho_imagem_busca = '/content/drive/MyDrive/catalogo_imagens/image_ebeafd.png' # <--- EDITE ESTE CAMINHO
num_recomendacoes = 5

print("--- Sistema de Recomendação Visual ---")

# 1. Extrair as features da imagem de busca
features_busca = extrair_features(caminho_imagem_busca, model)

if features_busca is not None:
    # 2. Calcular a similaridade de cosseno entre a busca e todos os itens do catálogo
    similaridades = cosine_similarity(features_busca.reshape(1, -1), vetores_features)

    # 3. Obter os índices das N imagens mais similares
    indices_similares = np.argsort(similaridades[0])[::-1]

    # --- Exibir os Resultados ---
    # Exibir a imagem de busca
    print("IMAGEM DE BUSCA:")
    try:
        plt.imshow(Image.open(caminho_imagem_busca))
        plt.axis('off')
        plt.show()

        # Exibir as recomendações
        print(f"\nTOP {num_recomendacoes} PRODUTOS RECOMENDADOS:")
        fig, axes = plt.subplots(1, num_recomendacoes, figsize=(20, 5))
        
        # Começamos de 1 se a imagem de busca estiver no catálogo, para não recomendá-la
        start_index = 0
        if caminho_imagem_busca in caminhos_imagens:
            start_index = 1

        for i, idx in enumerate(indices_similares[start_index : num_recomendacoes + start_index]):
            img_path = caminhos_imagens[idx]
            axes[i].imshow(Image.open(img_path))
            axes[i].set_title(f'Similaridade: {similaridades[0][idx]:.2f}')
            axes[i].axis('off')
        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        print(f"Erro: Imagem de busca não encontrada em '{caminho_imagem_busca}'.")
else:
    print("Não foi possível extrair features da imagem de busca.")